In [ ]:
!pip install --quiet --upgrade ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
import requests
import json

with open("/root/.kaggle/kaggle.json") as f:
    cred = json.load(f)


Status: 200
Response: [{"subtitleNullable":"Great for regression, classification, EDA, visualization, and even ML practice.","creatorNameNullable":"Jayanta Nath","creatorUrlNullable":"jayaantanaath","totalBytesNullable":19512,"urlNullable":"https://www.kaggle.com/datasets/jayaantanaath/student-habits-vs-academic-performance","licenseNameNullable":"Apache 2.0","descriptionNullable":null,"ownerNameNullable":"Jayanta Nath","ownerRefNullable":"jayaantanaath","titleNullable":"Student Habits vs Academic Performance","curre


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = cred['username']
os.environ['KAGGLE_KEY'] = cred['key']


In [ ]:
import kagglehub

path = kagglehub.dataset_download("bulatsharipov1/defect-detection-dataset")


print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/bulatsharipov1/defect-detection-dataset/versions/1


Path to the dataset


```
"/root/.cache/kagglehub/datasets/bulatsharipov1/defect-detection-dataset/versions/1/dataset_0_100"
```



In [ ]:
import random, yaml, os, re
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2

In [ ]:
def norm2abs(x_norm, y_norm, w, h):
    # в YOLO формате координаты отнормированы по размеру изображения
    # поэтому конвертируем их обратно в реальные координаты
    x_abs = x_norm * w
    y_abs = y_norm * h
    return np.column_stack((x_abs, y_abs))

In [ ]:
def clip_poly(poly, x0, y0, tile):
    # короче проверяет какие объекты в рассматриваемом подфрейме
    # Берет координаты маски, вычитает из них фрейм на котором ща находимся (чтобы отнормировать
    # координаты относительно нашего фрейма), а потом проверяет, входит ли маска в границы, если да
    # то кидаем
    # Кстати, тут по сути не будет потери масок из-за того что у нас пересечения, но все же могут быть
    poly[:, 0] -= x0
    poly[:, 1] -= y0
    inside = (poly[:, 0] >= 0) & (poly[:, 0] <= tile) &  (poly[:, 1] >= 0) & (poly[:, 1] <= tile)
    return poly[inside]

In [ ]:
def crop_the_dataset(dataset_orig_src_path, dataset_crop_dst_path, delete_original = True):
    # Функция, которая берет исходный датасет, и кропит изображение в несколько других изображений
    SRC = Path(dataset_orig_src_path)
    DST = Path(dataset_crop_dst_path)
    DST_IMG = DST / 'images'
    DST_LBL = DST / 'labels'
    for p in [DST_IMG, DST_LBL]: p.mkdir(parents=True, exist_ok=True)
    # parents = True: если родительских директорий не существует, создаем их
    # exist_ok = True: если папка уже существует, то оставляем как есть


    TILE = 1140 # Размер подизображения
    STRIDE = int(TILE * 0.8) # Чтобы были пересечения

    for img_path in tqdm(sorted((SRC / 'images').glob('*'))):
        h_img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
        # Здесь еще можно добавить проверку что количество измерений 3, чтобы если было 2, сделать РГБ
        # РГБ по сути не нужно в нашей таске, но все модельки работают именно с РГБ, поэтому терпим
        h, w = h_img.shape[:2]

        lbl_path = SRC / 'labels' / (img_path.stem + '.txt')
        polys_raw = open(lbl_path, 'r').read().strip().splitlines()
        # Здесь читаем файл с масками, каждая маска - новая линия, в каждой линии класс и координаты

        objs = []
        # Здесь будут храниться каждый класс с координатами

        for ln in polys_raw:
            # Собираем objs
            parts = ln.strip().split()
            cls = int(parts[0])
            xy_norm = np.array(parts[1:], float).reshape(-1, 2)
            objs.append((cls, xy_norm))


        id_tile = 0
        y0 = 0
        # id_tile - айдишник подизображения
        for x0 in range(0, w - TILE + 1, STRIDE):
            # прохожусь с окном TILE
            x1, y1 = x0 + TILE, y0 + TILE
            sub = h_img[y0:y1, x0:x1]

            new_lines = []
            for cls, xy_norm in objs:
                poly = norm2abs(xy_norm[:, 0], xy_norm[:, 1], w, h)
                # из нормированных координат получаем обычные
                poly = clip_poly(poly.copy(), x0, y0, TILE)
                # берет только те объекты которые в нашем изображении
                if len(poly) < 3:
                    # если меньше 3 точек то сегмент не получится
                    continue
                # обратно нормируем и добавляем
                poly_norm  = np.stack([poly[:, 0]/TILE, poly[:, 1]/TILE], 1).reshape(-1)
                poly_norm = ' '.join(f'{v:.6f}' for v in poly_norm)
                new_lines.append(f'{cls} {poly_norm}')
            # сохраняем под изображение
            out_stub = f'{img_path.stem}_{id_tile:04d}'
            cv2.imwrite(str(DST_IMG / f'{out_stub}.png'), sub)
            with open(DST_LBL / f'{out_stub}.txt', 'w' ) as f:
                if new_lines:
                    f.write('\n'.join(new_lines))
                else:
                    f.write('')
            id_tile += 1

        if delete_original:
            os.remove(img_path)
            os.remove(SRC / 'labels' / (img_path.stem + '.txt'))


In [ ]:
crop_the_dataset('/root/.cache/kagglehub/datasets/bulatsharipov1/defect-detection-dataset/versions/1/dataset_0_100', '/content/dataset_0_100', False )


100%|██████████| 938/938 [54:08<00:00,  3.46s/it]


In [ ]:
DATA_ROOT = Path('/content/dataset_0_100')
WORK_DIR  = DATA_ROOT

IMAGES_DIR = DATA_ROOT / 'images'
LABELS_DIR = DATA_ROOT / 'labels'

images = sorted(IMAGES_DIR.glob('*.png'))
random.shuffle(images)

split_ratio = 0.80
n_train = int(len(images) * split_ratio)
train_imgs, val_imgs = images[:n_train], images[n_train:]

def write_list(paths, file_path, root):
    with open(file_path, 'w') as f:
        for p in paths:
            rel = p.relative_to(root)
            f.write(rel.as_posix() + '\n')

TRAIN_LIST = WORK_DIR / 'train.txt'
VAL_LIST   = WORK_DIR / 'val.txt'
write_list(train_imgs, TRAIN_LIST, DATA_ROOT)
write_list(val_imgs,   VAL_LIST,   DATA_ROOT)

orig = yaml.safe_load(open(DATA_ROOT / 'data.yaml'))
names = {int(k): v for k, v in orig['names'].items()}

data_yaml_content = {
    'path': '.',
    'nc':   len(names),
    'names': names,
    'train': TRAIN_LIST.name,
    'val':   VAL_LIST.name
}

with open(WORK_DIR / 'data.yaml', 'w') as f:
    yaml.dump(data_yaml_content, f, allow_unicode=True)

print('✅ Prepared data.yaml\n', yaml.dump(data_yaml_content, allow_unicode=True))


✅ Prepared data.yaml
 names:
  0: пора
  1: включение
  2: подрез
  3: прожог
  4: трещина
  5: наплыв
  6: эталон1
  7: эталон2
  8: эталон3
  9: пора-скрытая
  10: утяжина
  11: несплавление
  12: непровар корня
nc: 13
path: .
train: train.txt
val: val.txt



In [ ]:
%cd /content/dataset_0_100
!kaggle datasets init -p .


/content/dataset_0_100
Data package template written to: ./dataset-metadata.json


In [ ]:
import json

meta_path = "dataset-metadata.json"
meta = json.load(open(meta_path))

meta["id"]          = "bulatsharipov1/defect-detection-cropped"
meta["title"]       = "cropped_version_hackath_innodsik"
meta["licenses"]    = [{"name": "CC0-1.0"}]

with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)


In [ ]:
!kaggle datasets create \
    -p . \
    --dir-mode zip

Starting upload for file labels.zip
100% 6.65M/6.65M [00:00<00:00, 29.1MB/s]
Upload successful: labels.zip (7MB)
Starting upload for file val.txt
100% 203k/203k [00:00<00:00, 1.17MB/s]
Upload successful: val.txt (203KB)
Starting upload for file train.txt
100% 812k/812k [00:00<00:00, 4.18MB/s]
Upload successful: train.txt (812KB)
Starting upload for file data.yaml
100% 336/336 [00:00<00:00, 2.02kB/s]
Upload successful: data.yaml (336B)
Starting upload for file images.zip
100% 21.6G/21.6G [03:18<00:00, 117MB/s]
Upload successful: images.zip (22GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/bulatsharipov1/defect-detection-cropped
